In [1]:
import pickle
import time
start_time = time.time()
import numpy as np
import os
import SimpleITK as sitk
import json
import ProstateLesionDetectionUtils
import random
import plotly.express as px
import matplotlib.pyplot as plt
from scipy import ndimage
import math
import tensorflow_addons as tfa
import tensorflow as tf
with open(r"E:\DIMITRIS_DEEP_LEARNING\DATASETS\UC1_Nifti_Datasets_Paths\Datasets.json", 'r') as f:
    datasets = json.load(f)
p158 = r"E:\DIMITRIS_DEEP_LEARNING\DATASETS\PROSTATE158\DATASETPICKLE\Nifti_files"
picai = r"E:\DIMITRIS_DEEP_LEARNING\DATASETS\PICAI_DATA\Nifti_files"
prostatenet = r"E:\DIMITRIS_DEEP_LEARNING\DATASETS\PROSTATENET_UC1_LESIONS\nnUNet_raw\Dataset011_ProstateNet"
with open(r"E:\DIMITRIS_DEEP_LEARNING\DATASETS\PROSTATENET_UC1_LESIONS\nnUNet_raw\Dataset011_ProstateNet\folds.json", 'r') as f:
    ValidationFolds = json.load(f)
    

def PrepareTraining(pat):
    dataset, labels = {}, {}
    for key in pat.keys():
        dataset.update({key:np.transpose(np.stack([pat[key]["T2"], pat[key]["ADC"], pat[key]["DWI"]]),(1,2,3,0))}) # tf.convert_to_tensor
        labels.update({key:np.expand_dims(pat[key]["Lesions"], axis = -1)}) # tf.convert_to_tensor
    return dataset, labels

c:\Users\dimza\anaconda3\envs\TensorFlow27\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (None)/charset_normalizer (3.1.0) doesn't match a supported version!
  warnings.warn(


In [2]:
loader = ProstateLesionDetectionUtils.DatasetsLoaders.Nifti_Loader.LoadSitkImages(ptdc = datasets["ProstateNet"], folder = prostatenet)
patients = loader.load_npobj()

In [ ]:
vis = ProstateLesionDetectionUtils.DatasetsProcessing.Visualization.Visualizer(patients)
vis.np_vis("DWI", "PCa-324365095643606010864476831387746614101")

In [ ]:
dt = ProstateLesionDetectionUtils.DetectionMetrics.ModelDetoriention.DetorientLesions(patients, rates = [15,40,50])
dt.CreateZones()
dt.CreateZaxisZone()
dt.KeepZonesNonZeroImageValues()
zones = dt.GetZones()
zaxis = dt.GetZaxisZones()

In [3]:
dataset, labels = PrepareTraining(patients)

x_train = {key:dataset[key] for key in dataset.keys() if key not in ValidationFolds["Fold 0"]}
y_train = {key:labels[key] for key in labels.keys() if key not in ValidationFolds["Fold 0"]}

x_test = {key:dataset[key] for key in dataset.keys() if key in ValidationFolds["Fold 0"]}
y_test = {key:labels[key] for key in labels.keys() if key in ValidationFolds["Fold 0"]}

In [ ]:
len(x_train),len(y_train),len(x_test),len(y_test)

In [4]:
Xtrain = list(x_train.values())
Ytrain = list(y_train.values())
Xtest = list(x_test.values())
Ytest = list(y_test.values())

In [5]:
start_time = time.time()
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
FILTERS = [32,64,128,256, 512, 1024, 2048]
Pool_size = ((2,2,2),(2,2,2),(2,2,2), (1,2,2), (1,2,2), (1,2,2))
VOLSIZE = 24
INPSIZE = (24,192,192,3)
BATCH_SIZE=1
EPOCHS = 100
INIT_LR = 1e-5
MAX_LR = 1e-2
TQDM_CB=tfa.callbacks.TQDMProgressBar()

METRICS = "acc" # tfa.metrics.F1Score(num_classes=2, threshold=0.5)

STEPS_PER_EPOCH = len(Xtrain) // BATCH_SIZE # len(train_data)

# CLR = lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
#     0.001,
#     decay_steps=STEPS_PER_EPOCH,  # Usually set to total number of training steps/epochs
#     decay_rate=0.96,
#     staircase=True
#)
CLR = tfa.optimizers.CyclicalLearningRate(initial_learning_rate=INIT_LR,
    maximal_learning_rate=MAX_LR,
    scale_fn=lambda x: 1/(2.**(x-1)),
    step_size=2 * STEPS_PER_EPOCH)
LOSS = tfa.losses.sigmoid_focal_crossentropy  #tfa.losses.sigmoid_focal_crossentropy
OPTIMIZER = tf.keras.optimizers.Adam(CLR)

name_folder = "TransUnet_8translayers_8heads"
try:
    os.mkdir(os.path.join(r"E:\DIMITRIS_DEEP_LEARNING\RESULTS\PROCANCER_UC1_RESULTS",name_folder))
except:
    pass

CHECK_CB = tf.keras.callbacks.ModelCheckpoint(os.path.join(r"E:\DIMITRIS_DEEP_LEARNING\RESULTS\PROCANCER_UC1_RESULTS",name_folder+"\CheckPoint_Weights.tf"),save_weights_only=False,save_best_only=True)
#EARLY_STOP = tf.keras.callbacks.EarlyStopping(patience = 30)
CALLBACKS=[CHECK_CB,TQDM_CB]#
#train_dataset = tf.data.Dataset.from_tensor_slices((Xtrain, Ytrain))
#val_dataset = tf.data.Dataset.from_tensor_slices((Xtest, Ytest))


# train_dataset = train_dataset.batch(BATCH_SIZE)
# val_dataset = val_dataset.batch(BATCH_SIZE)

TrainParams = {"LOSS":LOSS, "OPTIMIZER":OPTIMIZER, "CALLBACKS":CALLBACKS,"POOL_SIZE":Pool_size,
               "BATCH_SIZE":BATCH_SIZE, "FILTERS":FILTERS, "INPUT_SIZE":INPSIZE, "METRICS":METRICS,
               "WEIGHTS":os.path.join(r"E:\DIMITRIS_DEEP_LEARNING\RESULTS\PROCANCER_UC1_RESULTS",name_folder+"\CheckPoint_Weights.tf"),
               "VOLUME_SIZE":24, "EPOCHS":EPOCHS, "TRAIN_DATASET":{"DATA":Xtrain,"LABELS":Ytrain},"VAL_DATASET":{"DATA":Xtest,"LABELS":Ytest},"DENSE_SIZE":8, "PATCH_SIZE_XY": 2,
                "PATCH_SIZE_Z": 1,
                "TRANSF_HEADS": 4,
                "TRANSF_LAYERS": 8,
                "DROP_RATE": 0.4}

print((time.time() - start_time))

620.6974353790283


In [ ]:
start_time = time.time()
TrainParams["POOL_SIZE"] = ((2,2,2),(2,2,2),(2,2,2), (1,2,2), (1,2,2), (1,2,2))
TrainParams["PATCH_SIZE_XY"] = 3
mod = ProstateLesionDetectionUtils.DetectionModels.Models3D.TransUnet3D.transunet3d.TrainTransUnet(TrainParams)
mod.ModelBuild()
mod.ModelCompile()
#mod.LoadTrained()
mod.ModelFit()
#history = mod.GetHistory()
transunet = mod.GetModel()
transunet.save(os.path.join(r"E:\DIMITRIS_DEEP_LEARNING\RESULTS\PROCANCER_UC1_RESULTS",name_folder+"\Final_2048_patch_3.tf"))
print((time.time() - start_time))

# Evaluation

In [ ]:
name_folder = "TransUnet_8translayers_8heads"
TrainParams = {"LOSS":LOSS, "OPTIMIZER":OPTIMIZER, "CALLBACKS":CALLBACKS,"POOL_SIZE":Pool_size,
               "BATCH_SIZE":BATCH_SIZE, "FILTERS":FILTERS, "INPUT_SIZE":INPSIZE, "METRICS":METRICS,
               "WEIGHTS":os.path.join(r"E:\DIMITRIS_DEEP_LEARNING\RESULTS\PROCANCER_UC1_RESULTS",name_folder+"\CheckPoint_Weights.tf"),
               "VOLUME_SIZE":24, "EPOCHS":EPOCHS, "TRAIN_DATASET":train_dataset,"VAL_DATASET":val_dataset,"DENSE_SIZE":8, "PATCH_SIZE_XY": 2,
                "PATCH_SIZE_Z": 1,
                "TRANSF_HEADS": 8,
                "TRANSF_LAYERS": 8,
                "DROP_RATE": 0.5}

start_time = time.time()
mod = ProstateLesionDetectionUtils.DetectionModels.Models3D.TransUnet3D.transunet3d.TrainTransUnet(TrainParams)
mod.ModelBuild()
mod.ModelCompile()
mod.LoadTrained()
#mod.ModelFit()
#history = mod.GetHistory()
transunet = mod.GetModel()

print((time.time() - start_time))

In [ ]:
name_folder = "TransUnet_8translayers_8heads"
transunet = tf.keras.models.load_model(os.path.join(r"E:\DIMITRIS_DEEP_LEARNING\RESULTS\PROCANCER_UC1_RESULTS",name_folder+"\Final.tf"),compile = False)

In [ ]:
transunet.summary()

In [ ]:
#y_pred = {}
for key in x_test.keys():
    pred = tf.squeeze(tf.squeeze(transunet.predict(tf.expand_dims(x_test[key], axis = 0)), axis= 0), axis=-1).numpy()
    patients[key]["Pred"] = pred

    # {key:tf.squeeze(tf.squeeze(transunet.predict(tf.expand_dims(x_test[key], axis = 0)), axis= 0), axis=-1)}

In [ ]:
x_test.keys()

In [ ]:
ls = []
for key in y_test.keys():
    ls.append(sg.write_metrics(labels = [0,1], gdth_img = np.squeeze(y_test[key].numpy(), axis=-1), pred_img = y_pred[key].numpy()))


In [ ]:
x_test.keys()

In [ ]:
vis = ProstateLesionDetectionUtils.DatasetsProcessing.Visualization.Visualizer(patients)
vis.np_vis("Pred", 'PCa-271751484858119487045408524161595913267', alpha=0.3)